### Loading data using data loader

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 6.2 MB/s eta 0:00:00


In [ ]:
!pip install transformers --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.5 MB/s eta 0:00:00


In [ ]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
dataset = load_dataset("McGill-NLP/FaithDial")

Extracting data files:   0%|          | 0/7 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/3539 [00:00<?, ? examples/s]

Generating test_random_split split:   0%|          | 0/1716 [00:00<?, ? examples/s]

Generating test_topic_split split:   0%|          | 0/1823 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/18357 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3417 [00:00<?, ? examples/s]

Generating valid_random_split split:   0%|          | 0/1666 [00:00<?, ? examples/s]

Generating valid_topic_split split:   0%|          | 0/1751 [00:00<?, ? examples/s]

Dataset faith_dial downloaded and prepared to /root/.cache/huggingface/datasets/McGill-NLP___faith_dial/plain_text/1.0.0/70568c8ab3bbc83b603bce58fa593ab27e7f0d0cde51034e1c2073ff3e14189a. Subsequent calls will reuse this data.


  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
dataset.shape

{'test': (3539, 7),
 'test_random_split': (1716, 7),
 'test_topic_split': (1823, 7),
 'train': (18357, 7),
 'validation': (3417, 7),
 'valid_random_split': (1666, 7),
 'valid_topic_split': (1751, 7)}

In [ ]:
dataset["train"]

Dataset({
    features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
    num_rows: 18357
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [ ]:
from collections import Counter
from typing import List, Tuple
import datasets
import torch
from tqdm import tqdm

In [ ]:
dataset['train']

Dataset({
    features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
    num_rows: 18357
})

In [ ]:
class DatasetMap():
  @staticmethod
  def faithdial_begin(example):
    # nr_answer = len(example["response"])
    return [example["knowledge"]], [example["history"][-1]], [example["original_response"]], [','.join(example["BEGIN"])]

  @staticmethod
  def faithdial_vrm(example):
    # nr_answer = len(example["response"])
    return [example["knowledge"]], [example["history"]], [example["response"]], [','.join(example["VRM"])]
  
  @staticmethod
  def faithdial_comb(example):
    # nr_answer = len(example["response"])
    return [example["knowledge"]], [example["history"]], [example["response"]], [','.join(example["BEGIN"])], [','.join(example["VRM"])]

In [ ]:
class DialogDataset(torch.utils.data.Dataset):
  def __init__(self, hf_dataset: datasets.arrow_dataset.Dataset, tokenizer, parser=None):
        self.tokenizer = tokenizer
        self.questions: List[str] = []
        self.answers: List[str] = []
        self.contexts: List[str] = []
        self.begin: List[str] = []
        self.vrm: List[str] = []
        for row in tqdm(hf_dataset):
          _contexts, _questions, _answers, _begin, _vrm = parser(row)
          self.contexts += _contexts
          self.questions += _questions
          self.answers += _answers
          self.begin += _begin
          self.vrm += _vrm
          self.item_count: int = len(self.questions)

  def __len__(self):
      """Magic method over-ride for class lenght evaluation
      Returns:
          int: lenght of the object 
      """
      return self.item_count

  def __getitem__(self, index: int):
      """Magic method over-ride for class getitem method
      Args:
          index (int): index for identify question-context and answer example
      Returns:
          Tuple(str,str,str): (Context, Question, Answer)
      """
      return self.contexts[index], self.questions[index], self.answers[index], self.begin[index], self.vrm[index]

  def pack_minibatch(self, data: List[Tuple[str, str]]):
      """Pack mini-batch function
      Args:
          data (Tuple[List[str],List[str],List[str]]): (Contexts, Questions, Answers)
      Returns:
          Tuple[List[str],List[str],List[str]]: (Contexts, Questions, Answers)
      """
      return zip(*data)

  def __exact_match_score(self, prediction, ground_truth):
      """_summary_
      Args:
          prediction (_type_): _description_
          ground_truth (_type_): _description_
      Returns:
          _type_: _description_
      """
      if len(ground_truth) == len(prediction):
          if all(token1 == token2 for token1, token2 in zip(ground_truth,prediction)):
              return 1
      return 0

  def __f1_score(self, prediction_tokens, ground_truth_tokens):
      """_summary_
      Args:
          prediction (_type_): _description_
          ground_truth (_type_): _description_
      Returns:
          _type_: _description_
      """
      common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
      num_same = sum(common.values())
      if num_same == 0:
          return 0
      precision = 1.0 * num_same / len(prediction_tokens)
      recall = 1.0 * num_same / len(ground_truth_tokens)
      f1 = (2 * precision * recall) / (precision + recall)
      return f1

  def evaluate(self, predictions, gold_answers):
    f1 = exact_match = 0
    for ground_truths, prediction in tqdm(zip(gold_answers, predictions)):
        # Remove pad token
        tokens_to_remove = {
            self.tokenizer.pad_token_id,
            self.tokenizer.eos_token_id,
            self.tokenizer.bos_token_id,
            self.tokenizer.cls_token_id,
            self.tokenizer.sep_token_id,
            self.tokenizer.mask_token_id
          }
        prediction = list(filter(lambda token: token not in tokens_to_remove, prediction))
        ground_truths = list(filter(lambda token: token not in tokens_to_remove, ground_truths))
        f1 += self.__f1_score(prediction, ground_truths)
        exact_match += self.__exact_match_score(prediction, ground_truths)
    return 100*f1/len(predictions), 100*exact_match/len(predictions)

#### Using text to text custom pipeline for custom training loop

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_set = DialogDataset(dataset['train'], tokenizer, DatasetMap.faithdial_comb)

100%|██████████| 18357/18357 [00:04<00:00, 4485.81it/s]


In [ ]:
validation_set = DialogDataset(dataset['validation'], tokenizer, DatasetMap.faithdial_comb)

100%|██████████| 3417/3417 [00:00<00:00, 5024.43it/s]


In [ ]:
import random
print(random.choice(train_set))
print(random.choice(validation_set))

("But it can now be found on Shakur's compilation of ''Greatest Hits'' and on the United Kingdom version of ''All Eyez on Me''.", ['Have you ever heard of the song california love?', "No, I'm a bot and can't hear. But I know it was released by 2Pac right after he left prison in 1995.", "Yes, Tupac, I was trying to remember who wrote it. Weren't there some other people on it, as well?", 'Yes, it also featured Dr. Dre along with Roger Troutman.', 'Yeah! Now I remember it. I was just little when it came out, but my sister listened to it all the time.', 'The song California Love actually reached number one on the Billboard Hot 100. It stayed there two weeks.', 'I believe it. It was a hugely popular song.', "California Love was originally supposed to be on Dr Dre's album. That got scrapped though.", 'Dre is  smart man. He recognizes talent and makes a profit off of it. You got to respect him for that.'], "Yes, though you can find it on Shakur's Greatest Hits album.", 'Hallucination,Entailme

In [ ]:
BATCH_SIZE = 32

In [ ]:
my_trainset_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE,
                                    num_workers=8, collate_fn=lambda data: train_set.pack_minibatch(data))
my_validation_dataloader = DataLoader(validation_set, batch_size=BATCH_SIZE,
                                        num_workers=8, collate_fn=lambda data: validation_set.pack_minibatch(data))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


### Model Initialization

In [ ]:
# Initialize the T5 model
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

### Training

In [ ]:
import torch
from tqdm import tqdm

In [ ]:
DEVICE = 'cuda:0'
NUM_TRAIN_EPOCHS = 15
MAX_INPUT_LENGTH = 256

In [ ]:
model.train()
model.to(DEVICE)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
# Initialize the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [ ]:
for epoch in range(NUM_TRAIN_EPOCHS):
    epoch_train_loss = 0.
    ### Training Piece
    for contexts, questions, answers, begin, vrm in tqdm(my_trainset_dataloader):
        optimizer.zero_grad()
        inputs = list(map(lambda t: f"context: {t[0]}  question: {t[1]}  answer: {t[2]}", zip(contexts, questions, answers)))
        targets = list(map(lambda t: f"begin: {t[0]}  vrm: {t[1]}", zip(begin, vrm)))
        encoded_inputs = tokenizer(
                                inputs,
                                padding="longest",
                                max_length=MAX_INPUT_LENGTH,
                                truncation=True,
                                return_tensors="pt",
                            )
        
        encoded_targets = tokenizer(
                                targets,
                                padding="longest",
                                max_length=128,
                                truncation=True,
                                return_tensors="pt",
                            )

        input_ids, attention_mask = encoded_inputs.input_ids, encoded_inputs.attention_mask
        encoded_targets = encoded_targets.input_ids

        # replace padding target token id's of the labels by -100, crossEntropy skip target label == -100
        encoded_targets[encoded_targets == tokenizer.pad_token_id] = -100

        input_ids = input_ids.to(DEVICE)
        encoded_targets = encoded_targets.to(DEVICE)
        attention_mask = attention_mask.to(DEVICE)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=encoded_targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item() * BATCH_SIZE
    print(f"epoch={epoch + 1}/{NUM_TRAIN_EPOCHS}")
    print(f"\t Train loss = {epoch_train_loss/len(train_set):.4f}")

    ## Validation Piece
    model.eval()
    epoch_val_loss = 0.
    with torch.no_grad():
        # model_predictions_encoded = []
        # target_encoded = []
        for contexts, questions, answers, begin, vrm in tqdm(my_validation_dataloader):
            inputs = list(map(lambda t: f"context: {t[0]}  question: {t[1]}  answer: {t[2]}", zip(contexts, questions, answers)))
            targets = list(map(lambda t: f"begin: {t[0]}  vrm: {t[1]}", zip(begin, vrm)))
            encoded_inputs = tokenizer(
                inputs,
                padding="longest",
                max_length=MAX_INPUT_LENGTH,
                truncation=True,
                return_tensors="pt",
            )
            encoded_targets = tokenizer(
                targets,
                padding="longest",
                max_length=128,
                truncation=True,
                return_tensors="pt",
            )
            encoded_inputs, attention_mask = encoded_inputs.input_ids, encoded_inputs.attention_mask
            encoded_targets = encoded_targets.input_ids

            encoded_inputs = encoded_inputs.to(DEVICE)
            encoded_targets = encoded_targets.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
            outputs = model(input_ids=encoded_inputs, attention_mask=attention_mask, labels=encoded_targets)
            loss = outputs.loss
            epoch_val_loss += loss.item() * BATCH_SIZE
    print(f"\t Val loss = {epoch_val_loss/len(validation_set):.4f}")
    model.train()

100%|██████████| 574/574 [04:50<00:00,  1.97it/s]


epoch=1/15
	 Train loss = 0.4410


100%|██████████| 107/107 [00:19<00:00,  5.53it/s]


	 Val loss = 8.8421


100%|██████████| 574/574 [05:00<00:00,  1.91it/s]


...

	 Val loss = 9.5297


100%|██████████| 574/574 [05:02<00:00,  1.90it/s]


epoch=15/15
	 Train loss = 0.1350


100%|██████████| 107/107 [00:19<00:00,  5.44it/s]

	 Val loss = 9.8218


In [ ]:
model.save_pretrained("t5_finetuned_faithdial_qa_ep15_seqlen256")

### Inferencing

In [ ]:
test_sample = dataset['test'][20]

In [ ]:
test_sample

{'dialog_idx': 4,
 'response': 'Nice, did you know that elements of game design can also be applied to other interactions?',
 'original_response': 'Same and Increasingly, elements and principles of game design are also applied to other interactions',
 'history': ['I design video games for a living. Do you play any video games?',
  "I don't play any, but I do know about game design. It involves applying aesthetics and design to create a game.",
  'Yes! I enjoy making sports and strategy games. I love the way players caninteract on a sports game in a simulated environment.'],
 'knowledge': 'Increasingly, elements and principles of game design are also applied to other interactions, particularly virtual ones (see gamification).',
 'BEGIN': ['Hallucination', 'Entailment'],
 'VRM': ['Disclosure', 'Ack.']}

In [ ]:
test_sample

{'dialog_idx': 4,
 'response': 'Nice, did you know that elements of game design can also be applied to other interactions?',
 'original_response': 'Same and Increasingly, elements and principles of game design are also applied to other interactions',
 'history': ['I design video games for a living. Do you play any video games?',
  "I don't play any, but I do know about game design. It involves applying aesthetics and design to create a game.",
  'Yes! I enjoy making sports and strategy games. I love the way players caninteract on a sports game in a simulated environment.'],
 'knowledge': 'Increasingly, elements and principles of game design are also applied to other interactions, particularly virtual ones (see gamification).',
 'BEGIN': ['Hallucination', 'Entailment'],
 'VRM': ['Disclosure', 'Ack.']}

In [ ]:
def infer(model, prompt, knowledge, orig_resp, max_length=256):
    # question = "What is 42?"
    # context = "42 is the answer to life, the universe and everything"
    input = f"question: {prompt} context: {knowledge} answer: {orig_resp}"
    encoded_input = tokenizer([input],
                                return_tensors='pt',
                                max_length=256,
                                truncation=True)
    inp = encoded_input.input_ids.to(DEVICE)
    attn_mask = encoded_input.attention_mask.to(DEVICE)
    output = model.generate(input_ids = inp,
                                attention_mask = attn_mask, max_length=max_length)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    # print(output)
    return output

In [ ]:
targets = infer(model, prompt=test_sample['history'][-1], knowledge=test_sample['knowledge'], orig_resp = test_sample['original_response'])

In [ ]:
# epoch 10 seqlen 256
print('p:', test_sample['history'][-1])
print('k:', test_sample['knowledge'])
print('org:', test_sample['original_response'])
print('final prediction: ', targets)
print('expected_vrm: ', test_sample["BEGIN"])
print("expected_vrm: ", test_sample["VRM"])

p: Yes! I enjoy making sports and strategy games. I love the way players caninteract on a sports game in a simulated environment.
k: Increasingly, elements and principles of game design are also applied to other interactions, particularly virtual ones (see gamification).
org: Same and Increasingly, elements and principles of game design are also applied to other interactions
final prediction:  begin: Entailment vrm: Edification
expected_vrm:  ['Hallucination', 'Entailment']
expected_vrm:  ['Disclosure', 'Ack.']


In [ ]:
# Epoch 10 seqlen 512
print('p:', test_sample['history'][-1])
print('k:', test_sample['knowledge'])
print('org:', test_sample['original_response'])
print('final prediction: ', targets)
print('expected_vrm: ', test_sample["BEGIN"])
print("expected_vrm: ", test_sample["VRM"])

p: Yes! I enjoy making sports and strategy games. I love the way players caninteract on a sports game in a simulated environment.
k: Increasingly, elements and principles of game design are also applied to other interactions, particularly virtual ones (see gamification).
org: Same and Increasingly, elements and principles of game design are also applied to other interactions
final prediction:  begin: Entailment vrm: Edification
expected_vrm:  ['Hallucination', 'Entailment']
expected_vrm:  ['Disclosure', 'Ack.']


In [ ]:
ground_truths = test_sample["VRM"]

### Evaluating the model 

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(dataset['train']["VRM"])
num_labels = len(mlb.classes_)

In [ ]:
mlb.classes_

array(['', ' Advisement', ' Edification', ' edification', 'Ack.',
       'Advisement', 'Disclosure', 'Edification', 'Question'],
      dtype=object)

In [ ]:
classes = []
for i in range(1, len(mlb.classes_)):
  word = mlb.classes_[i]
  fword = word.strip().lower()

  if fword not in classes:
    classes.append(fword)
classes

['advisement', 'edification', 'ack.', 'disclosure', 'question']

In [ ]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(dataset['train']["BEGIN"])
num_labels = len(mlb.classes_)

In [ ]:
mlb.classes_

array(['Entailment', 'Generic', 'Hallucination', 'Uncooperative'],
      dtype=object)

In [ ]:
for i in mlb.classes_:
  classes.append(i.strip().lower())

In [ ]:
classes

['advisement',
 'edification',
 'ack.',
 'disclosure',
 'question',
 'entailment',
 'generic',
 'hallucination',
 'uncooperative']

In [ ]:
yhat = []
for i in range(len(dataset['test'])):
  n = []
  print("example", i)
  targets = infer(model, knowledge=dataset['test']['knowledge'][i], prompt = dataset['test']['history'][i][-1], orig_resp = dataset['test']['original_response'][i])
  targets = targets.split(",")
  token = []
  for word in targets:
    token.append(word.strip().lower())
  for j in range(len(classes)):
    if classes[j] in token:
      n.append(1)
    else:
      n.append(0)
  yhat.append(n)

example 0
example 1
example 2
example 3
example 4
example 5
example 6
example 7
example 8
example 9
example 10
example 11
example 12
example 13
example 14
example 15
example 16
example 17
example 18
example 19
example 20
example 21
example 22
example 23
example 24
example 25
example 26
example 27
example 28
example 29
example 30
example 31
example 32
example 33
example 34
example 35
example 36
example 37
example 38
example 39
example 40
example 41
example 42
example 43
example 44
example 45
example 46
example 47
example 48
example 49
example 50
example 51
example 52
example 53
example 54
example 55
example 56
example 57
example 58
example 59
example 60
example 61
example 62
example 63
example 64
example 65
example 66
example 67
example 68
example 69
example 70
example 71
example 72
example 73
example 74
example 75
example 76
example 77
example 78
example 79
example 80
example 81
example 82
example 83
example 84
example 85
example 86
example 87
example 88
example 89
example 90
example 9

In [ ]:
y = []
for i in range(len(dataset['test'])):
  n = []
  exp = dataset['test']["VRM"][i] + dataset['test']["BEGIN"][i]
  real = []
  for word in exp:
    real.append(word.strip().lower())
  for j in range(len(classes)):
    if classes[j] in real:
      n.append(1)
    else:
      n.append(0)
  y.append(n)
print(y)

[[0, 1, 0, 1, 1, 0, 0, 1, 0], [0, 1, 0, 0, 0, 1, 0, 0, 1], [0, 1, 0, 0, 0, 1, 0, 0, 0], [0, 1, 1, 0, 0, 1, 0, 1, 0], [0, 1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 0, 0, 0, 1, 0, 1, 0], [0, 1, 0, 0, 0, 1, 0, 1, 0], [0, 1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 0, 1, 1, 1, 0, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 0], [0, 1, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0, 1, 0], [0, 1, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 1, 0, 1, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 0], [0, 0, 1, 1, 0, 1, 0, 1, 0], [0, 1, 0, 0, 0, 1, 0, 0, 0], [0, 1, 1, 0, 0, 1, 0, 0, 0], [0, 0, 1, 1, 0, 1, 0, 1, 0], [0, 1, 1, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 1, 1, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0, 0, 0], [0, 1, 1, 1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 1, 

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
f1 = []
accuracy = []
for i in range(len(yhat)):
  macro_f1 = f1_score(yhat[i], y[i], average='macro')
  f1.append(macro_f1)
  acc = accuracy_score(y[i], yhat[i])
  accuracy.append(acc)
print(f1)
print(accuracy)
print("Macro-Averaged F1 Score:", sum(f1)/len(f1))
print("Average accuracy", sum(accuracy)/len(accuracy))

[0.5846153846153846, 0.6785714285714286, 0.6785714285714286, 0.5846153846153846, 0.35714285714285715, 0.35714285714285715, 0.35714285714285715, 0.35714285714285715, 0.25, 0.35714285714285715, 0.4, 0.6785714285714286, 0.4, 0.43750000000000006, 0.43750000000000006, 0.8, 0.8, 0.5846153846153846, 0.43750000000000006, 0.35714285714285715, 0.3076923076923077, 0.43750000000000006, 0.6785714285714286, 0.3076923076923077, 0.5, 0.75, 0.8, 0.5846153846153846, 0.43750000000000006, 0.39999999999999997, 0.8, 0.5846153846153845, 0.8, 0.4, 0.43750000000000006, 0.6785714285714286, 0.75, 0.6785714285714286, 0.6785714285714286, 0.39999999999999997, 0.25, 0.4, 0.6785714285714286, 0.35714285714285715, 0.35714285714285715, 0.35714285714285715, 0.4, 0.4, 0.43750000000000006, 0.4, 0.5846153846153846, 0.6785714285714286, 0.6785714285714286, 0.4, 0.5, 0.6785714285714286, 0.3076923076923077, 0.3076923076923077, 0.3076923076923077, 0.35714285714285715, 0.4, 0.30769230769230765, 0.4, 0.3076923076923077, 0.30769230

In [ ]:
print(sum(conf)/ len(yhat))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
confusion_matrix = multilabel_confusion_matrix(y, yhat)
for i, cm in enumerate(confusion_matrix):
    print(f"Confusion matrix for class {i + 1}:\n{cm}\n")

Confusion matrix for class 1:
[[3409    0]
 [ 128    2]]

Confusion matrix for class 2:
[[ 947  550]
 [1333  709]]

Confusion matrix for class 3:
[[1534  702]
 [ 659  644]]

Confusion matrix for class 4:
[[1661    0]
 [1878    0]]

Confusion matrix for class 5:
[[3033  131]
 [ 293   82]]

Confusion matrix for class 6:
[[ 860    0]
 [2679    0]]

Confusion matrix for class 7:
[[3507    0]
 [  32    0]]

Confusion matrix for class 8:
[[ 978    0]
 [2561    0]]

Confusion matrix for class 9:
[[3329    0]
 [ 210    0]]



In [ ]:
confusion_matrix??

In [ ]:
classes

['advisement',
 'edification',
 'ack.',
 'disclosure',
 'question',
 'entailment',
 'generic',
 'hallucination',
 'uncooperative']